## 네이버 검색 api를 활용해 핀테크 관련 뉴스를 매일 수집하는 실행파일 생성

In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import os
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from dbio import er_to_db, db_connect
from datetime import datetime

In [2]:
def string_delete(text) :
    return text.replace("<b>","").replace("</b>", "").replace("&quot;", "")

def parse_date(date_str):
    # 날짜 문자열을 datetime 객체로 변환
    return datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S %z")

def make_df(df_name) :
    for i in df_name :
        for idx in range(0, len(df_name[i])) : 
            df_name[i][idx] = string_delete(df_name[i][idx])
    df_name['pubDate'] = df_name['pubDate'].apply(parse_date)
    return df_name
    

In [9]:
client_id = os.getenv('client_id') # 환경 변수에 접근
client_pw = os.getenv('client_pw') # 환경 변수에 접근

url = "https://openapi.naver.com/v1/search/news.json"
payload = {'query' : '핀테크', 'display' : 10, 'start' : 1, 'sort' : 'date'}
headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_pw}
r = requests.get(url, params = payload, headers = headers)
if r.status_code == 200 :
    data = r.json()
#     print(data)
else :
    print("Error Code:", r.status_code)
    

df = pd.json_normalize(data['items'])
make_df(df)

C:\Users\5-08\AppData\Local\Temp\ipykernel_16372\3190665787.py:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_name[i][idx] = string_delete(df_name[i][idx])


,title,originallink,link,description,pubDate
0,"서울대 경영·경제학 전공자들, 법무법인 도아 설립",https://news.heraldcorp.com/view.php?ud=202408...,https://n.news.naver.com/mnews/article/016/000...,"오수비 대표변호사는 법무법인(유한) 태평양·화우에서 블록체인, 핀테크, 컴플라이언스...",2024-08-02 11:23:00+09:00
1,"롯데 칼리버스, 8월 오픈 앞두고 글로벌 광폭 행보",http://www.economytalk.kr/news/articleView.htm...,http://www.economytalk.kr/news/articleView.htm...,"김동규 칼리버스 대표이사, 롯데이노베이트 고두영 대표이사, 신유열 롯데지주 미래성장...",2024-08-02 11:16:00+09:00
2,"서비스 오픈 앞둔 롯데 메타버스 '칼리버스', 日 라쿠텐과 사업 협력",https://news.mtn.co.kr/news-detail/20240802103...,https://news.mtn.co.kr/news-detail/20240802103...,"김동규 칼리버스 대표, 고두영 롯데이노베이트 대표, 신유열 롯데지주 미래성장실장, ...",2024-08-02 11:14:00+09:00
3,"우리투자증권, 그룹 효자 계열사 될까…우선과제 살펴보니",http://news.tf.co.kr/read/economy/2121538.htm,https://n.news.naver.com/mnews/article/629/000...,"우리투자증권이 연말까지 자체 MTS를 선보인다는 계획을 내세웠지만, 토스증권 등 핀...",2024-08-02 11:08:00+09:00
4,"한투파트너스, '핀테크혁신펀드' 1호 상장 일궜다",https://www.newstof.com/news/articleView.html?...,https://www.newstof.com/news/articleView.html?...,한국투자금융지주 계열 벤처캐피탈인 한국투자파트너스가 핀테크혁신펀드 위탁운용사(GP)...,2024-08-02 11:00:00+09:00
5,[금융가 이모저모] 롤 판에 등장한 'BNK',https://news.einfomax.co.kr/news/articleView.h...,https://news.einfomax.co.kr/news/articleView.h...,핀테크·플랫폼 기업이 금융의 영역으로 진입하면서 전통 은행들도 젊은 금융소비자들에게...,2024-08-02 10:58:00+09:00
6,"카카오페이, 명동 소상공인과 함께하는 ‘써머 페스타 in 명동’ 후원",http://www.segye.com/newsView/20240802508403?O...,https://n.news.naver.com/mnews/article/022/000...,카카오페이는 핀테크사 중 유일하게 외국인 관광객들이 한국에 왔을 때 카카오페이 가맹...,2024-08-02 10:49:00+09:00
7,"은행, 동남아에 K-금융 심는다",https://www.fetv.co.kr/news/article.html?no=17...,https://www.fetv.co.kr/news/article.html?no=17...,웹케시글로벌은 B2B(기업과 기업 간 거래) 핀테크 전문기업 웹케시그룹의 글로벌 사...,2024-08-02 10:32:00+09:00
8,"뱅크웨어글로벌, 코스닥 상장…한국투자파트너스 투자",https://dealsite.co.kr/articles/126324,https://dealsite.co.kr/articles/126324,회사는 한국투자파트너스가 2020년 결성한 핀테크혁신펀드의 포트폴리오로 투자 3년 ...,2024-08-02 10:14:00+09:00
9,"미쓰비시UFJ은행, 필리핀 디지털 금융 대기업에 633억엔 출자",http://www.g-enews.com/ko-kr/news/article/news...,http://www.g-enews.com/ko-kr/news/article/news...,일본 금융 대기업 미쓰비시 UFJ 은행이 2일 필리핀의 디지털 금융 서비스 대기업인...,2024-08-02 09:44:00+09:00


In [7]:
now = datetime.now()
now

datetime.datetime(2024, 8, 2, 11, 25, 27, 140630)

In [10]:
filtered_df = df[df['pubDate'].dt.date == now.date()]
filtered_df

,title,originallink,link,description,pubDate
0,"서울대 경영·경제학 전공자들, 법무법인 도아 설립",https://news.heraldcorp.com/view.php?ud=202408...,https://n.news.naver.com/mnews/article/016/000...,"오수비 대표변호사는 법무법인(유한) 태평양·화우에서 블록체인, 핀테크, 컴플라이언스...",2024-08-02 11:23:00+09:00
1,"롯데 칼리버스, 8월 오픈 앞두고 글로벌 광폭 행보",http://www.economytalk.kr/news/articleView.htm...,http://www.economytalk.kr/news/articleView.htm...,"김동규 칼리버스 대표이사, 롯데이노베이트 고두영 대표이사, 신유열 롯데지주 미래성장...",2024-08-02 11:16:00+09:00
2,"서비스 오픈 앞둔 롯데 메타버스 '칼리버스', 日 라쿠텐과 사업 협력",https://news.mtn.co.kr/news-detail/20240802103...,https://news.mtn.co.kr/news-detail/20240802103...,"김동규 칼리버스 대표, 고두영 롯데이노베이트 대표, 신유열 롯데지주 미래성장실장, ...",2024-08-02 11:14:00+09:00
3,"우리투자증권, 그룹 효자 계열사 될까…우선과제 살펴보니",http://news.tf.co.kr/read/economy/2121538.htm,https://n.news.naver.com/mnews/article/629/000...,"우리투자증권이 연말까지 자체 MTS를 선보인다는 계획을 내세웠지만, 토스증권 등 핀...",2024-08-02 11:08:00+09:00
4,"한투파트너스, '핀테크혁신펀드' 1호 상장 일궜다",https://www.newstof.com/news/articleView.html?...,https://www.newstof.com/news/articleView.html?...,한국투자금융지주 계열 벤처캐피탈인 한국투자파트너스가 핀테크혁신펀드 위탁운용사(GP)...,2024-08-02 11:00:00+09:00
5,[금융가 이모저모] 롤 판에 등장한 'BNK',https://news.einfomax.co.kr/news/articleView.h...,https://news.einfomax.co.kr/news/articleView.h...,핀테크·플랫폼 기업이 금융의 영역으로 진입하면서 전통 은행들도 젊은 금융소비자들에게...,2024-08-02 10:58:00+09:00
6,"카카오페이, 명동 소상공인과 함께하는 ‘써머 페스타 in 명동’ 후원",http://www.segye.com/newsView/20240802508403?O...,https://n.news.naver.com/mnews/article/022/000...,카카오페이는 핀테크사 중 유일하게 외국인 관광객들이 한국에 왔을 때 카카오페이 가맹...,2024-08-02 10:49:00+09:00
7,"은행, 동남아에 K-금융 심는다",https://www.fetv.co.kr/news/article.html?no=17...,https://www.fetv.co.kr/news/article.html?no=17...,웹케시글로벌은 B2B(기업과 기업 간 거래) 핀테크 전문기업 웹케시그룹의 글로벌 사...,2024-08-02 10:32:00+09:00
8,"뱅크웨어글로벌, 코스닥 상장…한국투자파트너스 투자",https://dealsite.co.kr/articles/126324,https://dealsite.co.kr/articles/126324,회사는 한국투자파트너스가 2020년 결성한 핀테크혁신펀드의 포트폴리오로 투자 3년 ...,2024-08-02 10:14:00+09:00
9,"미쓰비시UFJ은행, 필리핀 디지털 금융 대기업에 633억엔 출자",http://www.g-enews.com/ko-kr/news/article/news...,http://www.g-enews.com/ko-kr/news/article/news...,일본 금융 대기업 미쓰비시 UFJ 은행이 2일 필리핀의 디지털 금융 서비스 대기업인...,2024-08-02 09:44:00+09:00


## 오늘 발행된 핀테크 관련 뉴스 데이터프레임으로 만들고 db에 저장하기

In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import os
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from dbio import news_db_connect, news_to_db
from datetime import datetime

In [2]:
def string_delete(text) :
    return text.replace("<b>","").replace("</b>", "").replace("&quot;", "")

def parse_date(date_str):
    # 날짜 문자열을 datetime 객체로 변환
    return datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S %z")

def make_df(df_name) :
    for i in df_name :
        for idx in range(0, len(df_name[i])) : 
            df_name[i][idx] = string_delete(df_name[i][idx])
    df_name['pubDate'] = df_name['pubDate'].apply(parse_date)
    return df_name

client_id = os.getenv('client_id') # 환경 변수에 접근
client_pw = os.getenv('client_pw') # 환경 변수에 접근

url = "https://openapi.naver.com/v1/search/news.json"
payload = {'query' : '핀테크', 'display' : 100, 'start' : 1, 'sort' : 'date'}
headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_pw}
r = requests.get(url, params = payload, headers = headers)
if r.status_code == 200 :
    data = r.json()
else :
    print("Error Code:", r.status_code)
    
df = pd.json_normalize(data['items'])

# 제목과 요약에 태그 지우고 날짜 데이터 타입 바꾸기
make_df(df)

# 오늘 날짜 생성
now = datetime.now()

# 날짜 비교 후 오늘 날짜인 뉴스만 남기기
filtered_df = df[df['pubDate'].dt.date == now.date()]

news_to_db('fintech_news',now.date(), filtered_df)

C:\Users\5-08\AppData\Local\Temp\ipykernel_10140\300177682.py:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_name[i][idx] = string_delete(df_name[i][idx])
